In [3]:
import os
import sys
import shutil 
sys.path.append('/scratch/users/k21066795/prj_normal/RandStainNA')
from randstainna import RandStainNA

import glob
import h5py
import random
from PIL import Image
from tqdm import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch


from randstainna import RandStainNA
import cv2

from utils_data import add_ageGroup
from utils_features import ensemble_feas_labels, labelingFeature, CSVDataset_WSI, get_model, extract_features, sampeled_UMP
from utils import plot_multiple, plot_oneline

import shutil
# dir_name = '/scratch/prj/cb_normalbreast/Siyuan/prj_normal/BreastAgeNet/DATA/bagFeatures/development'
# shutil.make_archive(dir_name, 'zip', dir_name)

#--------------------------check clinic----------------------------#

In [98]:
cls = ['epithelium', 'stroma'] # _patch.csv only to get patches predicted as cls
WSIs = '/scratch/prj/cb_normalbreast/Siyuan/prj_normal/BreastAgeNet/WSIs/SGK_healthy'  # where to get WSIs???
rootdir = '/scratch/prj/cb_normalbreast/Siyuan/prj_normal/BreastAgeNet/RootDir/SGK_healthy' # where to get _patch.csv, .h5 files
clinic = '/scratch/prj/cb_normalbreast/Siyuan/prj_normal/BreastAgeNet/DATA/clinicData' # where to get clinic.csv

In [99]:
# get KHP_RM df
SGK_clinic = glob.glob(f"{clinic}/*SGK*")[0]

SGK_clinic_df = pd.read_csv(SGK_clinic)
SGK_clinic_df = add_ageGroup(SGK_clinic_df) # add categorical age group based on patients' age

_, counts = np.unique(SGK_clinic_df['age_group'], return_counts=True)
m, y, o = counts
print(f"percentage of patients in each group \n <30y: {y}  30-50y: {m}  >50y {o}")

percentage of patients in each group 
 <30y: 268  30-50y: 853  >50y 1087


#--------------------------extract features----------------------------#

In [ ]:
model_name = 'resnet50'
WSIs = '/scratch/prj/cb_normalbreast/Siyuan/prj_normal/BreastAgeNet/WSIs/KHP_RM'  # where to get WSIs???
rootdir = '/scratch/prj/cb_normalbreast/Siyuan/prj_normal/BreastAgeNet/RootDir/KHP_RM' # where to get _patch.csv, .h5 files
cls = ['epithelium', 'stroma'] # _patch.csv only to get patches predicted as cls
batch_size = 4
num_workers = 0

In [ ]:
# get GPU
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(f"device: {device}")
# get the model
model, transform = get_model(model_name, device)
print(f"model_name: {model_name}")
for wsiname in os.listdir(rootdir):
    bag_folder = os.path.join(rootdir, wsiname)
    # define the output .h5 feature file
    fname = f"{bag_folder}/{os.path.basename(bag_folder)}_bagFeature_{model_name}.h5"
    if not os.path.exists(fname):
        bag_csv = glob.glob(f"{bag_folder}/*_patch.csv")  # localise the file containing patch ids
        if len(bag_csv)>0:
            bag_df = pd.read_csv(bag_csv[0])
            if len(bag_df) >0:
                print(f"------extracting {model_name} features for {wsiname}------")
                bag_df = bag_df.loc[bag_df['cls'].isin(cls)]
                # give the bag_df containing patch ids, folder containing WSIs, model, output file pt
                extract_features(bag_df, WSIs, model, transform, device, fname, batch_size=4, num_workers=0)

###### here, the issue was when submitting jobs, can not handle multiple processes

In [6]:
rein_files_tCpoy = []

for model_name in ['gigapath', 'UNI', 'iBOT', 'resnet50']:
    feature_files = glob.glob(f"{rootdir}/*/*_bagFeature_*{model_name}_reinhard.h5")
    print(f"{model_name}, {len(feature_files)}")
    rein_files_tCpoy.extend(feature_files)
    
print(len(rein_files_tCpoy))

gigapath, 915
UNI, 915
iBOT, 915
resnet50, 915
3660


In [5]:
aug_files_tCpoy = []
for model_name in ['gigapath', 'UNI', 'iBOT', 'resnet50']:
    feature_files = glob.glob(f"{rootdir}/*/*_bagFeature_*{model_name}_augmentation.h5")
    print(f"{model_name}, {len(feature_files)}")
    aug_files_tCpoy.extend(feature_files)

print(len(aug_files_tCpoy))

gigapath, 915
UNI, 915
iBOT, 915
resnet50, 915
3660


'\ning...: \ngigapath norm x2\ngiga aug \nUNI aug \niBOT norm\nres norm\nres aug x2\n'

#-------------------------- feature transfer ----------------------------#

In [9]:
import shutil 


# for src in rein_files_tCpoy:
#     basename = os.path.basename(src)
#     model_name, stainFunc = basename.split('_')[-2:]
#     stainFunc = stainFunc.split('.')[0]
    
#     tar_folder = f'/scratch/prj/cb_normalbreast/Siyuan/prj_normal/BreastAgeNet/DATA/bagFeatures/development/{model_name}_{stainFunc}'
#     tar = os.path.join(tar_folder, basename)

#     shutil.copyfile(src, tar)


# for src in aug_files_tCpoy:
#     basename = os.path.basename(src)
#     model_name, stainFunc = basename.split('_')[-2:]
#     stainFunc = stainFunc.split('.')[0]
    
#     tar_folder = f'/scratch/prj/cb_normalbreast/Siyuan/prj_normal/BreastAgeNet/DATA/bagFeatures/development/{model_name}_{stainFunc}'
#     tar = os.path.join(tar_folder, basename)

#     shutil.copyfile(src, tar)

#-------------------------- feature and clinic ----------------------------#

In [35]:
# merge_feature_clinic(rootdir, clinic=os.path.join(clinic, 'SGK_2k.csv'))
ages = []
for wsiname in os.listdir(rootdir):
    folder = os.path.join(rootdir, wsiname)
    pt = glob.glob(f'{folder}/*patch_merge.csv')
    if len(pt) >0:
        pt = pt[0]
        df = pd.read_csv(pt)
        if len(df)>0:
            ages.append(np.unique(df['age_group'])[0])

_, counts = np.unique(ages, return_counts=True)
m, y, o = counts
print(f"percentage of patients in each group \n <30y: {y}  30-50y: {m}  >50y {o}")

percentage of patients in each group 
 <30y: 162  30-50y: 412  >50y 341


#--------------------------show umap----------------------------#

In [41]:
def show_Umap_norm_aug(model_name, rootdir, sample_num,n=50, d=0.2):
    imgs = []
    captions = []
    
    stainFunc = 'reinhard'
    meta_label = 'cls'
    img = sampeled_UMP(model_name, meta_label, sample_num, rootdir, stainFunc, n, d, save_pt=None)
    imgs.append(img)
    captions.append(f'{model_name}_{stainFunc}_cls')
    
    meta_label = 'age_group'
    img = sampeled_UMP(model_name, meta_label, sample_num, rootdir, stainFunc, n, d, save_pt=None)
    imgs.append(img)
    captions.append(f'{model_name}_{stainFunc}_age_group')
    
    stainFunc = 'augmentation'
    meta_label = 'cls'
    img = sampeled_UMP(model_name, meta_label, sample_num, rootdir, stainFunc, n, d, save_pt=None)
    imgs.append(img)
    captions.append(f'{model_name}_{stainFunc}_cls')
    
    meta_label = 'age_group'
    img = sampeled_UMP(model_name, meta_label, sample_num, rootdir, stainFunc, n, d, save_pt=None)
    imgs.append(img)
    captions.append(f'{model_name}_{stainFunc}_age_group')
    
    plot_multiple(imgs, ['norm','norm','aug','aug'], 2,2 ,figure_size=(5,5), cmap='tab20')

In [42]:
model_name = 'iBOT'
sample_num = 50000

show_Umap_norm_aug(model_name, rootdir, sample_num)

(309152, 768)


TypeError: '<' not supported between instances of 'float' and 'NoneType'

<Figure size 1000x1000 with 0 Axes>

In [ ]:
model_name = 'UNI'
sample_num = 50000

show_Umap_norm_aug(model_name, rootdir, sample_num)

#--------------------------Examples of stain augmentation ----------------------------#

In [ ]:
"""
Usage1: Demo(for visualization)
"""
augmentor = RandStainNA(
    yaml_file = '../RandStainNA/CRC_LAB_randomTrue_n0.yaml',
    std_hyper = 0.0,
    distribution = 'normal',
    probability = 1.0,
    is_train = True
)
# img:is_train:false——>np.array()(cv2.imread()) #BGR
# img:is_train:True——>PIL.Image #RGB

print(augmentor)

In [ ]:
img_path_list = [
    '../RandStainNA/visualization/origin/TUM-AEPINLNQ.png',
    '../RandStainNA/visualization/origin/TUM-DFGFFNEY.png',
    '../RandStainNA/visualization/origin/TUM-EWFNFSQL.png',
    '../RandStainNA/visualization/origin/TUM-TCGA-CVATFAAT.png'
]

origimgs = []
for i in img_path_list:
    origimgs.append(Image.open(i))

plot_multiple(origimgs, ['','','',''], 2,2 ,figure_size=(5,5), cmap='tab20')

In [ ]:
augimgs = []

for img_path in img_path_list:
    img = augmentor(Image.open(img_path))
    augimgs.append(img)
    
plot_multiple(augimgs, ['','','',''], 2,2 ,figure_size=(5,5), cmap='tab20')

#-------------------------- prepare external testing ----------------------------#

In [90]:
rootdir = '/scratch/prj/cb_normalbreast/Siyuan/prj_normal/BreastAgeNet/RootDir/KHP_RM' # containing .h5 features
clinic = '/scratch/prj/cb_normalbreast/Siyuan/prj_normal/BreastAgeNet/DATA/KHP_clinic.csv' # containing age group
stainFunc = 'normalisation'

In [91]:
files = []
for wsiname in os.listdir(rootdir):
    folder = os.path.join(rootdir, wsiname)
    pt = glob.glob(f"{folder}/*_TC*mask.npy")
    if len(pt)>0:
        files.append(pt[0])

len(files)

219

In [92]:
files = []
for wsiname in os.listdir(rootdir):
    folder = os.path.join(rootdir, wsiname)
    pt = glob.glob(f"{folder}/*_patch.csv")
    if len(pt)>0:
        files.append(pt[0])

len(files)

219

In [95]:
files = []
for wsiname in os.listdir(rootdir):
    folder = os.path.join(rootdir, wsiname)
    pt = glob.glob(f"{folder}/*_reinhard.h5")
    if len(pt)>0:
        files.extend(pt)

len(files)

197

In [77]:
rootdir = '/scratch/prj/cb_normalbreast/Siyuan/prj_normal/BreastAgeNet/RootDir/KHP_RRM' # containing .h5 features
clinic = '/scratch/prj/cb_normalbreast/Siyuan/prj_normal/BreastAgeNet/DATA/KHP_clinic.csv' # containing age group
stainFunc = 'normalisation'

In [86]:
files = []
for wsiname in os.listdir(rootdir):
    folder = os.path.join(rootdir, wsiname)
    pt = glob.glob(f"{folder}/*_TC*mask.npy")
    if len(pt)>0:
        files.append(pt[0])

len(files)

66

In [87]:
files = []
for wsiname in os.listdir(rootdir):
    folder = os.path.join(rootdir, wsiname)
    pt = glob.glob(f"{folder}/*_patch.csv")
    if len(pt)>0:
        files.append(pt[0])

len(files)

0

In [82]:
rootdir = '/scratch/prj/cb_normalbreast/Siyuan/prj_normal/BreastAgeNet/RootDir/NKI' # containing .h5 features
clinic = '/scratch/prj/cb_normalbreast/Siyuan/prj_normal/BreastAgeNet/DATA/clinicData/NKI_clinic.csv' # containing age group
stainFunc = 'normalisation'

In [83]:
files = []
for wsiname in os.listdir(rootdir):
    folder = os.path.join(rootdir, wsiname)
    pt = glob.glob(f"{folder}/*_TC*mask.npy")
    if len(pt)>0:
        files.append(pt[0])

len(files)

156

In [84]:
rootdir = '/scratch/prj/cb_normalbreast/Siyuan/prj_normal/BreastAgeNet/RootDir/BCI' # containing .h5 features
clinic = '/scratch/prj/cb_normalbreast/Siyuan/prj_normal/BreastAgeNet/DATA/clinicData/NKI_clinic.csv' # containing age group
stainFunc = 'normalisation'

In [85]:
files = []
for wsiname in os.listdir(rootdir):
    folder = os.path.join(rootdir, wsiname)
    pt = glob.glob(f"{folder}/*_TC*mask.npy")
    if len(pt)>0:
        files.append(pt[0])

len(files)

66